# Aprendizado Profundo - UFMG

## Sem MXNET

Observe que este notebook não faz uso de mxnet. Vamos implementar um mini-arcabouço de redes neurais para entender um pouco ferramentas tipo mxnet e pytorch por baixo.

In [0]:
import numpy as np


# TESTEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE

## Mini Introdução OO

Uma grande parte das bibliotecas de hoje em dia exploram conceitos de orientação a objetos. Embora não tempos tempo de ensinar um curso inteiro de OO, seria pelo menos um semestre, podemos cobrir o essencial neste notebook. Caso queira uma única palavra para sumarizar: o essencial aqui é o conceito de **estado**!.

Existem dois termos importantes para entender OO:
1. Classes
2. Objetos

Classes definem um esqueleto do que será armazenado. Objetos definem uma instância da classe na memória. Abaixo temos uma classe simples chamadas de agregador. No momento, a mesma não faz nada!

In [0]:
class Agregador(object):
    pass

Podemos instanciar nossas classes, ficando assim com dois objetos na memória:

In [0]:
obj1 = Agregador()
obj2 = Agregador()

Cada instância, ou objeto, difere um do outro

In [0]:
obj1 == obj2

False

Agora, vamos adicionar um atríbuto na nossa classe. O mesmo guarda um valor simples. Se objetos são instâncias de classes, algum local tem guardar o estado desta instância. Em python, tal local é chamado de **self**. Todo objeto tem uma referência para seu estado, o self, que no fim das contas é uma referência para ele mesmo. 

In [0]:
class Agregador(object):
    def __init__(self):
        self._value = 0   # Toda instância agora guarda um objeto simples!

In [0]:
obj1 = Agregador()
obj2 = Agregador()

print(obj1._value)
print(obj2._value)

0
0


Dois objetos podem guardar um mesmo estado. Acima, os dois tem valor = 0. Ainda assim, são dois objetos diferentes, ocupam locais diferentes da memória do computador.

In [0]:
hex(id(obj1))

'0x7f0de6449400'

In [0]:
hex(id(obj2))

'0x1076defd0'

In [0]:
obj3 = obj1
hex(id(obj3)) # note como aqui temos o mesmo endereco do obj1

'0x1076fc048'

Agora tornar os nossos objetos um pouco mais inteligentes. Vamos adicionar um método, nome bonito para uma função, que altera o valor do atributo value.

In [0]:
class Agregador(object):
    
    def __init__(self):
        self._value = 0 
    
    def adiciona(self, value):
        self._value += value
        
    def get_value(self):
        return self._value

In [0]:
agg = Agregador()

Observe como temos uma chamada `get_value()`. 

In [0]:
print(agg.get_value())

0


Ao adicionar alguma coisa, temos um novo estado!

In [0]:
agg.adiciona(7)
print(agg.get_value())

7


Abaixo temos dois objetos diferentes com o mesmo estado. É isto!

In [0]:
agg2 = Agregador()
agg2.adiciona(agg.get_value())

print(agg == agg2)
print(agg.get_value() == agg2.get_value())

False
True


## Forward e Backward

Imagine que você tem uma memória onde você guarda algumas funções para derivar. Por clareza, o seu código de autograd, estimo o `mxnet`, só sabe derivar a forma $f(x, n) = x^n$. Vamos criar uma mini-biblioteca de autoderivar agora. A mesma é composta de classes que representam constantes. Vamos chamar isto de uma camada constante, para usar os termos que serão comuns em mxnet/pytorch.

In [0]:
class CamadaConstante(object):
    
    def __init__(self, valor):
        self.valor = valor
    
    def avalia(self):
        return self.valor
    
    def feedforward(self, camada_anterior):
        return self.valor
    
    def backpropagate(self):
        return 0

Observe como a mesma apenas retorna o valor no método `feedforward`. O mesmo representa o código executado dentro do autograd. Neste momento, avaliamos funções. Aqui, a função nada mais é do que uma constante.

Além do mais, temos um parâmetro no `__init__`. Tal método é uma chamada especial que inicializa o estado. Note no passo a passo [deste link](http://pythontutor.com/visualize.html#code=class%20CamadaConstante%28object%29%3A%0A%20%20%20%20%0A%20%20%20%20def%20__init__%28self,%20valor%29%3A%0A%20%20%20%20%20%20%20%20self.valor%20%3D%20valor%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20def%20feedforward%28self%29%3A%0A%20%20%20%20%20%20%20%20return%20self.valor%0A%20%20%20%20%0A%20%20%20%20def%20backpropagate%28self%29%3A%0A%20%20%20%20%20%20%20%20return%200%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%0Acamada%20%3D%20CamadaConstante%287%29%0Aprint%28camada.feedforward%28%29%29%0Aprint%28camada.backpropagate%28%29%29%0A%0Acamada%20%3D%20CamadaConstante%2812%29%0Aprint%28camada.feedforward%28%29%29%0Aprint%28camada.backpropagate%28%29%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=py3anaconda&rawInputLstJSON=%5B%5D&textReferences=false) como ela é chamada ao criar o objeto.

Note também como em algum momento temos dois objetos na memória com estados diferentes.

**Brinque com o passo a passo do link acima antes de continuar!**

.
.
.

Agora, vamos criar uma camada que cuida da forma: $f(x, n) = x^n$

In [0]:
class CamadaPolinomial(object):
    
    def __init__(self, n):
        self.n = n
    
    def avalia(self):
        return np.power(self.x, self.n)
    
    def feedforward(self, camada_anterior):
        self.x = camada_anterior.avalia()
        return np.power(self.x, self.n)
    
    def backpropagate(self):
        return self.n * self.x

A classe `CamadaPolinomial` cuida de derivar polinômios. Note que no método `feedforward` fazemos duas coisas:

1. Avaliamos uma constante
1. Guardamos o valor da mesma
1. Retornamos $f(x, n) = x^n$

In [0]:
rede_neural = CamadaPolinomial(7)
rede_neural.feedforward(CamadaConstante(9))

4782969

Parece que o código está correto!

In [0]:
np.power(9, 7)

4782969

Ao chamar o `backpropagate`, temos a derivada: $f'(x) = nx$

In [0]:
rede_neural.backpropagate()

63

In [0]:
7 * 9

63

A ideia da derivação automágica do mxnet é mais ou menos essa. Diferente da brincadeira acima, a biblioteca é capaz de derivar bem mais do que polinômios e constantes.

Observe como abaixo usando uma lista simulamos a ideia de uma camada passando mensagens para outras. Caso necessite visualizar, o passo a passo está [aqui]().

In [0]:
rede_neural = []
rede_neural.append(CamadaConstante(7))
rede_neural.append(CamadaPolinomial(1))
rede_neural.append(CamadaPolinomial(9))

In [0]:
rede_neural[0].feedforward(7)
rede_neural[1].feedforward(rede_neural[0])
rede_neural[2].feedforward(rede_neural[1])
rede_neural[2].backpropagate()

63